In [ ]:

# This notebook is used to generate triplet of music attributes (melody) for lyrics in the test data.


In [2]:

import numpy as np
import tensorflow as tf
import midi_statistics
import utils
import pickle
import matplotlib.pyplot as plt


In [3]:
def do_generate_songs(model_path, data):
  generated_songs = []

  model_stats = {}
  model_stats['stats_scale_tot'] = 0
  model_stats['stats_repetitions_2_tot'] = 0
  model_stats['stats_repetitions_3_tot'] = 0
  model_stats['stats_span_tot'] = 0
  model_stats['stats_unique_tones_tot'] = 0
  model_stats['stats_avg_rest_tot'] = 0
  model_stats['num_of_null_rest_tot'] = 0
  model_stats['best_scale_score'] = 0
  model_stats['best_repetitions_2'] = 0
  model_stats['best_repetitions_3'] = 0
  model_stats['num_perfect_scale'] = 0
  model_stats['num_good_songs'] = 0
  model_stats['avg_songlength'] = 0

  with tf.compat.v1.Session(graph=tf.Graph()) as sess:
      tf.compat.v1.saved_model.loader.load(sess, [], model_path)
      graph = tf.compat.v1.get_default_graph()
      keep_prob = graph.get_tensor_by_name("model/keep_prob:0")
      input_metadata = graph.get_tensor_by_name("model/input_metadata:0")
      input_songdata = graph.get_tensor_by_name("model/input_data:0")
      output_midi = graph.get_tensor_by_name("output_midi:0")
      feed_dict = {}
      feed_dict[keep_prob.name] = 1
      condition = []
      feed_dict[input_metadata.name] = condition


      for i in range(len(data)):
          feed_dict[input_songdata.name] = np.random.uniform(size=(1, 20, 3))
          condition.append(np.split(data[i][3 * 20:], 20))
          feed_dict[input_metadata.name] = condition
          generated_features = sess.run(output_midi, feed_dict)
          sample = [x[0, :] for x in generated_features]
          sample = midi_statistics.tune_song(utils.discretize(sample))
          generated_songs.append(sample)
          condition = []

          # get stats
          stats = midi_statistics.get_all_stats(sample)
          model_stats['stats_scale_tot'] += stats['scale_score']
          model_stats['stats_repetitions_2_tot'] += float(stats['repetitions_2'])
          model_stats['stats_repetitions_3_tot'] += float(stats['repetitions_3'])
          model_stats['stats_unique_tones_tot'] += float(stats['tones_unique'])
          model_stats['stats_span_tot'] += stats['tone_span']
          model_stats['stats_avg_rest_tot'] += stats['average_rest']
          model_stats['num_of_null_rest_tot'] += stats['num_null_rest']
          model_stats['best_scale_score'] = max(stats['scale_score'], model_stats['best_scale_score'])
          model_stats['best_repetitions_2'] = max(stats['repetitions_2'], model_stats['best_repetitions_2'])
          model_stats['best_repetitions_3'] = max(stats['repetitions_3'], model_stats['best_repetitions_3'])
          model_stats['avg_songlength'] += stats['songlength']

          if stats['scale_score'] == 1.0 and stats['tones_unique'] > 3 \
              and stats['tone_span'] > 4 and stats['num_null_rest'] > 8 and stats['tone_span'] < 13\
              and stats['repetitions_2'] > 4:
              model_stats['num_good_songs'] += 1
      
      model_stats['avg_songlength'] /= len(data)

  return generated_songs, model_stats

In [8]:

train_data = np.load('./data/processed_dataset_matrices/train_data_matrix.npy')

train_data.shape


(11149, 460)

In [10]:
train_data[0,:60]

array([76.  ,  0.25,  0.  , 76.  ,  0.75,  0.  , 76.  ,  0.25,  0.  ,
       76.  ,  3.  ,  1.  , 72.  ,  1.  ,  8.  , 77.  ,  2.  ,  0.  ,
       76.  ,  1.  ,  0.  , 72.  ,  3.  ,  0.  , 76.  ,  1.  ,  8.  ,
       74.  ,  0.5 ,  0.  , 74.  ,  1.  ,  0.  , 72.  ,  4.  ,  0.  ,
       76.  ,  1.  ,  1.  , 74.  ,  1.  ,  0.  , 74.  ,  1.  ,  0.  ,
       72.  ,  4.  ,  0.  , 76.  ,  2.  ,  1.  , 74.  ,  0.75,  0.  ,
       74.  ,  1.  ,  0.  , 72.  ,  6.  ,  0.  ])

In [11]:
train_data[0,60:]

array([ 7.10338640e+00,  1.01343184e+01, -8.97774506e+00,  1.16764927e+00,
       -1.01303831e-01, -1.92353711e-01, -5.53583145e+00,  5.58647253e-02,
       -3.52456355e+00, -2.06991649e+00, -3.06762671e+00,  1.65503082e+01,
       -7.76225424e+00, -1.20424576e+01,  9.88166046e+00,  4.82475710e+00,
       -1.55747509e+01, -2.14776802e+00, -2.74804473e-01, -4.67991924e+00,
       -5.83118320e-01, -3.85189623e-01, -7.00767136e+00,  3.96358371e+00,
        2.61698842e+00, -2.98423290e-01,  1.41342327e-01,  3.52733791e-01,
        2.75696182e+00, -4.43324184e+00, -2.93441081e+00,  2.23827600e+00,
        1.28789127e+00, -1.71256351e+00, -3.00144887e+00, -2.18054795e+00,
       -3.14800715e+00,  4.40390635e+00, -3.03551364e+00, -6.88842773e-01,
       -3.20523620e-01, -2.64736629e+00, -7.11456680e+00,  3.66899800e+00,
        2.11378002e+00,  1.50973582e+00,  9.35248756e+00, -2.81228352e+00,
        2.32891154e+00, -2.23388910e+00, -1.48912983e+01,  4.85247552e-01,
        2.82039142e+00, -

In [4]:

test_data = np.load('./data/processed_dataset_matrices/test_data_matrix.npy')

test_data.shape


(1051, 460)

In [5]:
def get_all_stats_from_array(d_midi):
    _num_midi_features = 3
    _seq_length = int(len(d_midi) / _num_midi_features)
    stats = {}
    tones = []
    note_type = []
    rest = []
    for i in range(_seq_length):
        tones.append(d_midi[i*_num_midi_features+0])
    for i in range(_seq_length):
        note_type.append(d_midi[i*_num_midi_features+1])
    for i in range(_seq_length):
        rest.append(d_midi[i*_num_midi_features+2])
    if len(tones) == 0:
        print('This is an empty song.')
        return None
    stats['num_tones'] = len(tones)
    stats['tone_min'] = min(tones)
    stats['tone_max'] = max(tones)
    stats['tone_span'] = max(tones)-min(tones)
    stats['tones_unique'] = len(set(tones))
    rs = midi_statistics.repetitions(tones)
    for r in range(2, 10):
        if r in rs:
          stats['repetitions_{}'.format(r)] = rs[r]
        else:
          stats['repetitions_{}'.format(r)] = 0
    ml = midi_statistics.max_likelihood_scale(tones)
    stats['scale'] = ml[0]
    stats['scale_score'] = ml[1]
    stats['rest_max'] = max(rest)
    stats['average_rest'] = np.mean(rest)
    stats['num_null_rest'] = rest.count(0)
    stats['songlength'] = sum(rest) + sum(note_type)

    return stats

In [6]:
def analyze_ground_truth(data):
  # for each data[i], data[i][:60] is the midi data. the rest is the lyrics data
  model_stats = {}
  model_stats['stats_scale_tot'] = 0
  model_stats['stats_repetitions_2_tot'] = 0
  model_stats['stats_repetitions_3_tot'] = 0
  model_stats['stats_span_tot'] = 0
  model_stats['stats_unique_tones_tot'] = 0
  model_stats['stats_avg_rest_tot'] = 0
  model_stats['num_of_null_rest_tot'] = 0
  model_stats['best_scale_score'] = 0
  model_stats['best_repetitions_2'] = 0
  model_stats['best_repetitions_3'] = 0
  model_stats['num_perfect_scale'] = 0
  model_stats['num_good_songs'] = 0
  model_stats['avg_songlength'] = 0

  for d in data:
    d_midi = d[:60]
    stats = get_all_stats_from_array(d_midi)
    # get stats
    model_stats['stats_scale_tot'] += stats['scale_score']
    model_stats['stats_repetitions_2_tot'] += float(stats['repetitions_2'])
    model_stats['stats_repetitions_3_tot'] += float(stats['repetitions_3'])
    model_stats['stats_unique_tones_tot'] += float(stats['tones_unique'])
    model_stats['stats_span_tot'] += stats['tone_span']
    model_stats['stats_avg_rest_tot'] += stats['average_rest']
    model_stats['num_of_null_rest_tot'] += stats['num_null_rest']
    model_stats['best_scale_score'] = max(stats['scale_score'], model_stats['best_scale_score'])
    model_stats['best_repetitions_2'] = max(stats['repetitions_2'], model_stats['best_repetitions_2'])
    model_stats['best_repetitions_3'] = max(stats['repetitions_3'], model_stats['best_repetitions_3'])
    model_stats['avg_songlength'] += stats['songlength']

    if stats['scale_score'] == 1.0 and stats['tones_unique'] > 3 \
        and stats['tone_span'] > 4 and stats['num_null_rest'] > 8 and stats['tone_span'] < 13\
        and stats['repetitions_2'] > 4:
        model_stats['num_good_songs'] += 1
    
  model_stats['avg_songlength'] /= len(data)

  return model_stats

In [7]:
test_gt_stats = analyze_ground_truth(test_data)
valid_gt_stats = analyze_ground_truth(valid_data)
train_gt_stats = analyze_ground_truth(train_data)

NameError: name 'valid_data' is not defined

In [ ]:
test_gt_stats, valid_gt_stats, train_gt_stats

({'stats_scale_tot': 1003.4500000000007,
  'stats_repetitions_2_tot': 7803.0,
  'stats_repetitions_3_tot': 4039.0,
  'stats_span_tot': 11326.0,
  'stats_unique_tones_tot': 6184.0,
  'stats_avg_rest_tot': 815.2499999999987,
  'num_of_null_rest_tot': 16380,
  'best_scale_score': 1.0,
  'best_repetitions_2': 18,
  'best_repetitions_3': 17,
  'num_perfect_scale': 0,
  'num_good_songs': 383,
  'avg_songlength': 43.25404376784015},
 {'stats_scale_tot': 1004.9500000000018,
  'stats_repetitions_2_tot': 7889.0,
  'stats_repetitions_3_tot': 4086.0,
  'stats_span_tot': 10946.0,
  'stats_unique_tones_tot': 6155.0,
  'stats_avg_rest_tot': 830.8999999999995,
  'num_of_null_rest_tot': 16142,
  'best_scale_score': 1.0,
  'best_repetitions_2': 18,
  'best_repetitions_3': 17,
  'num_perfect_scale': 0,
  'num_good_songs': 416,
  'avg_songlength': 43.13796384395813},
 {'stats_scale_tot': 10642.79999999999,
  'stats_repetitions_2_tot': 83330.0,
  'stats_repetitions_3_tot': 43417.0,
  'stats_span_tot': 1170

In [ ]:
model_path = 'saved_gan_models/saved_model_best_overall_mmd'

In [ ]:
test_gen_songs, test_model_stats = do_generate_songs(model_path, test_data)

In [1]:
train_data = np.load('./data/processed_dataset_matrices/train_data_matrix.npy')
train_data.shape

NameError: name 'np' is not defined

In [ ]:
valid_data = np.load('./data/processed_dataset_matrices/valid_data_matrix.npy')
valid_data.shape

(1051, 460)

In [ ]:
valid_gen_songs, valid_model_stats = do_generate_songs(model_path, valid_data)

In [ ]:
train_gen_songs, train_model_stats = do_generate_songs(model_path, train_data)

In [ ]:
test_gen_songs = np.asarray(test_gen_songs)
test_gen_songs.shape

(1051, 20, 3)

In [ ]:
valid_get_songs = np.asarray(valid_gen_songs)
valid_get_songs.shape

(1051, 20, 3)

In [ ]:
train_get_songs = np.asarray(train_gen_songs)
train_get_songs.shape

(11149, 20, 3)

In [ ]:
test_model_stats

{'stats_scale_tot': 1051.0,
 'stats_repetitions_2_tot': 6694.0,
 'stats_repetitions_3_tot': 2630.0,
 'stats_span_tot': 11992.0,
 'stats_unique_tones_tot': 6805.0,
 'stats_avg_rest_tot': 338.50000000000006,
 'num_of_null_rest_tot': 17057,
 'best_scale_score': 1.0,
 'best_repetitions_2': 14,
 'best_repetitions_3': 12,
 'num_perfect_scale': 0,
 'num_good_songs': 582,
 'avg_songlength': 25.955280685061847}

In [ ]:
valid_model_stats

{'stats_scale_tot': 1051.0,
 'stats_repetitions_2_tot': 6616.0,
 'stats_repetitions_3_tot': 2619.0,
 'stats_span_tot': 12397.0,
 'stats_unique_tones_tot': 6941.0,
 'stats_avg_rest_tot': 347.1000000000005,
 'num_of_null_rest_tot': 16903,
 'best_scale_score': 1.0,
 'best_repetitions_2': 15,
 'best_repetitions_3': 13,
 'num_perfect_scale': 0,
 'num_good_songs': 551,
 'avg_songlength': 26.238582302568982}

In [ ]:
train_model_stats

{'stats_scale_tot': 11149.0,
 'stats_repetitions_2_tot': 70182.0,
 'stats_repetitions_3_tot': 27138.0,
 'stats_span_tot': 130290.0,
 'stats_unique_tones_tot': 73094.0,
 'stats_avg_rest_tot': 3476.5500000000447,
 'num_of_null_rest_tot': 182102,
 'best_scale_score': 1.0,
 'best_repetitions_2': 16,
 'best_repetitions_3': 14,
 'num_perfect_scale': 0,
 'num_good_songs': 6100,
 'avg_songlength': 25.76015786169163}

In [ ]:

generated_pitches = generated_songs[:, :, 0]
generated_lengths = generated_songs[:, :, 1]
generated_rests   = generated_songs[:, :, 2]

generated_pitches.shape, generated_lengths.shape, generated_rests.shape


((1051, 20), (1051, 20), (1051, 20))

In [ ]:

np.save(model_path + '/generated_songs',   generated_songs)
np.save(model_path + '/generated_pitches', generated_pitches)
np.save(model_path + '/generated_lengths', generated_lengths)
np.save(model_path + '/generated_rests',   generated_rests)
